In [ ]:
GAMING_CSV_PATH = "../Jarir-scraper/jarir_gaming_pcs.csv"
LAPTOP_CSV_PATH = "../Jarir-scraper/jarir_laptops.csv"
TABLET_CSV_PATH = "../Jarir-scraper/jarir_tablets.csv"
twoin1_CSV_PATH = "../Jarir-scraper/jarir_twoin1_laptops.csv"
DESKTOPS_CSV_PATH = "../Jarir-scraper/jarir_desktops.csv"
AIO_CSV_PATH = "../Jarir-scraper/jarir_AIO.csv"


csv_paths = [GAMING_CSV_PATH, LAPTOP_CSV_PATH, TABLET_CSV_PATH , twoin1_CSV_PATH , DESKTOPS_CSV_PATH, AIO_CSV_PATH]

import pandas as pd
from pathlib import Path
from typing import Dict, List

def build_brand_first_map(csv_paths: List[str]) -> Dict[str, Dict[str, List[str]]]:
    """
    Read each CSV at the given file paths (each must have 'brand' and 'model' cols),
    and merge into one dict mapping:

        brand → { product_type → sorted list of unique models }.

    The product_type is taken from the CSV filename stem, e.g. 'tablet' for 'tablet.csv'.
    """
    brand_first_map: Dict[str, Dict[str, List[str]]] = {}

    for path in csv_paths:
        # derive product_type from filename
        product_type = Path(path).stem
        if product_type.lower().endswith("_csv_path"):
            product_type = product_type[: -len("_csv_path")]
        product_type = product_type.lower()

        # read only the brand & model columns
        df = pd.read_csv(path, usecols=['brand', 'model'])

        # for each brand, collect its models under this product_type
        for brand in df['brand'].dropna().unique():
            models = (
                df.loc[df['brand'] == brand, 'model']
                  .dropna()
                  .unique()
                  .tolist()
            )
            models = sorted(models)
            # initialize brand entry if needed
            if brand not in brand_first_map:
                brand_first_map[brand] = {}
            # assign the list under this product_type
            brand_first_map[brand][product_type] = models

    return brand_first_map


brand_model_map = build_brand_first_map(csv_paths)

def retrieve_information_about_brand(brand: str) -> Dict[str, List[str]]:
    """
    Given a brand name, returns a dict mapping each product_type
    to the list of models available under that brand.
    If the brand is not found, returns an empty dict.
    """
    return brand_model_map.get(brand, {})


#--------------------------



def build_product_type_first_map(csv_paths: List[str]) -> Dict[str, Dict[str, List[str]]]:
    """
    Read each CSV at the given file paths (each must have 'brand' and 'model' cols),
    and merge into one dict mapping:

        product_type → { brand → sorted list of unique models }.

    product_type will be one of:
      ['gaming', 'laptop', 'tablet', 'twoin1_laptop', 'desktops','AIO']
    """
    nested_map: Dict[str, Dict[str, List[str]]] = {}

    for path in csv_paths:
        # 1) derive normalized product_type
        stem = Path(path).stem.lower()            # e.g. "jarir_gaming_pcs"
        if stem.startswith("jarir_"):
            stem = stem[len("jarir_"):]           # → "gaming_pcs"
        stem = stem.replace("_pcs", "")           # → "gaming"
        # singularize where needed:
        if stem.endswith("s") and stem not in ("desktops",):
            stem = stem[:-1]                      # "laptops"→"laptop", "tablets"→"tablet"
        product_type = stem                       # now one of gaming, laptop, tablet, twoin1, desktops

        # 2) read & group
        df = pd.read_csv(path, usecols=["brand", "model"])
        brand_map: Dict[str, List[str]] = {}
        for brand in df["brand"].dropna().unique():
            models = (
                df.loc[df["brand"] == brand, "model"]
                  .dropna()
                  .unique()
                  .tolist()
            )
            brand_map[brand] = sorted(models)

        nested_map[product_type] = brand_map

    return nested_map

product_type_map = build_product_type_first_map(csv_paths)

def retrieve_information_about_product_type(product_type: str) -> Dict[str, List[str]]:
    """
    These are the available product_type ['gaming', 'laptop', 'tablet', 'twoin1_laptop', 'desktops','AIO']
    Given a product type, returns a dict mapping each brand
    to the list of models available under that product_type.
    """
    return product_type_map.get(product_type.lower(), {})

In [32]:
print(retrieve_information_about_product_type("AIO"))

{'Asus': ['A3', 'AIO A5702'], 'Lenovo': ['IdeaCentre AIO', 'IdeaCentre AIO 3', 'Yoga AIO'], 'Acer': ['Aspire C'], 'HP': ['All-in-One'], 'Apple': ['iMac', 'iMac M3', 'iMac M4']}
